## Experiments: Sensitive Analysis Hyperparameter RSAST:


It is runned RSAST in a set of UCR datasets with a predefined number of runs ("runs"). Then, it is selected a range ("range_total") between 1, 10, 30 ,50 and 100 for the selected dataset.

In [1]:
#configure directory to import sast libraries
import sys 
import os 
#add sast library path
file_path = os.path.expanduser("~/random_sast/sast")
#file_path = r"C:\Users\Public\random_sast\sast"
sys.path.append(file_path)
file_path = os.path.expanduser("~\random_sast\sast")
#file_path = r"C:\Users\Public\random_sast\sast"
sys.path.append(file_path)

#add cd_diagram library path
file_path = os.path.expanduser("~/random_sast/cd_diagram")
#file_path = r"C:\Users\Public\random_sast\cd_diagram"
sys.path.append(file_path)
file_path = os.path.expanduser("~\random_sast\sast")
#file_path = r"C:\Users\Public\random_sast\sast"
sys.path.append(file_path)

sys.path

['c:\\Users\\Surface pro\\random_sast\\ExperimentationRSAST',
 'c:\\Users\\Surface pro\\anaconda3\\python39.zip',
 'c:\\Users\\Surface pro\\anaconda3\\DLLs',
 'c:\\Users\\Surface pro\\anaconda3\\lib',
 'c:\\Users\\Surface pro\\anaconda3',
 '',
 'c:\\Users\\Surface pro\\anaconda3\\lib\\site-packages',
 'c:\\Users\\Surface pro\\anaconda3\\lib\\site-packages\\win32',
 'c:\\Users\\Surface pro\\anaconda3\\lib\\site-packages\\win32\\lib',
 'c:\\Users\\Surface pro\\anaconda3\\lib\\site-packages\\Pythonwin',
 'c:\\Users\\Surface pro\\anaconda3\\lib\\site-packages\\IPython\\extensions',
 'C:\\Users\\Surface pro\\.ipython',
 'C:\\Users\\Surface pro/random_sast/sast',
 'C:\\Users\\\random_sast\\sast',
 'C:\\Users\\Surface pro/random_sast/cd_diagram',
 'C:\\Users\\\random_sast\\sast']

In [2]:
from sast import *
import pandas as pd
import researchpy
import math

c:\Users\Surface pro\anaconda3\lib\site-packages\mass_ts\_mass_ts.py:17: UserWarning: GPU support will not work. You must pip install mass-ts[gpu].
  warnings.warn(


### Reading Datasets

In [3]:
# Set directory where the csv files are located
directory = os.getcwd()+'/ResultsByClassifier'

# Create an empty list to store the dataframes
dfs = []

# Loop through all files in the directory
for filename in os.listdir(directory):
    # Check if the file starts with "df_overall_tunning" and ends with ".csv"
    if filename.endswith(".csv"):
        # Read the csv file into a dataframe
        filepath = os.path.join(directory, filename)
        df = pd.read_csv(filepath)
        # Append the dataframe to the list
        df['filename']=filename
        dfs.append(df)

# Concatenate all the dataframes in the list into one dataframe
df_other_methods = pd.concat(dfs, ignore_index=True)
df_other_methods=df_other_methods[['filename','folds:','0']]
df_other_methods['method']=df_other_methods['filename'].str.split('_').str[0]
df_other_methods
df_other_methods=df_other_methods.rename(columns={'folds:':'dataset','0':'score'})
df_other_methods=df_other_methods[['dataset','score','method']]


In [4]:
df_other_methods['method'].unique()

array(['BOSS', 'Catch22', 'cBOSS', 'HIVE-COTEv1', 'InceptionTime',
       'ProximityForest', 'ResNet', 'RISE', 'ROCKET', 'S-BOSS', 'SAST',
       'STC', 'TS-CHIEF', 'TSF', 'WEASEL'], dtype=object)

In [5]:
# Set directory where the csv files are located
directory = os.getcwd()+'/ResultsRsast'
#directory = os.getcwd()+'/results_accuracy_per_ds'

# Create an empty list to store the dataframes
dfs = []

# Loop through all files in the directory
for filename in os.listdir(directory):
    # Check if the file starts with "df_overall_tunning" and ends with ".csv"
    if filename.startswith("df_overall_tunning") and filename.endswith(".csv"):
        # Read the csv file into a dataframe
        filepath = os.path.join(directory, filename)
        df = pd.read_csv(filepath)
        df['rpoint']=df['rpoint'].astype(str)
        df['nb_per_class']=df['nb_per_class'].astype(str)

        
        # Append the dataframe to the list
        dfs.append(df)

# Concatenate all the dataframes in the list into one dataframe
df_result = pd.concat(dfs, ignore_index=True)
#df_result.head(3)


In [6]:
#create dataset to generate cd diagram
df_perf=pd.pivot_table(df_result, values='accuracy', index=['classifier_name','dataset_name'], aggfunc='mean')
df_perf=df_perf.reset_index()
#df_perf.classifier_name.unique()

In [7]:
#get ds tested overall
df_result['dataset_name'].unique()

array(['Chinatown', 'Coffee', 'DistalPhalanxOutlineAgeGroup',
       'DistalPhalanxOutlineCorrect', 'GunPoint', 'ItalyPowerDemand',
       'ShapeletSim', 'SmoothSubspace', 'SonyAIBORobotSurface2',
       'SyntheticControl'], dtype=object)

In [8]:
#get columns of generated datasets
df_result.columns.unique()

Index(['Unnamed: 0', 'accuracy', 'time', 'cweights_time', 'fsubsequence_time',
       'tdataset_time', 'tclassifier_time', 'dataset_name', 'classifier_name',
       'rpoint', 'nb_per_class', 'method', 'len_method'],
      dtype='object')

In [9]:
#filter dataset for cd-diagrams with the ds that has already tested all combinations of params
param_per_ds=pd.pivot_table(df_perf, index=['dataset_name'],values=['classifier_name'] ,aggfunc='count')
param_per_ds=param_per_ds.reset_index()
complete_ds=param_per_ds[param_per_ds.classifier_name==max(param_per_ds.classifier_name)]["dataset_name"].unique()

df_perf=df_perf[df_perf.dataset_name.isin(complete_ds)]
df_result=df_result[df_result.dataset_name.isin(complete_ds)]
df_perf.dataset_name.unique()

array(['Chinatown', 'Coffee', 'DistalPhalanxOutlineAgeGroup',
       'DistalPhalanxOutlineCorrect', 'GunPoint', 'ItalyPowerDemand',
       'SmoothSubspace', 'SonyAIBORobotSurface2', 'SyntheticControl'],
      dtype=object)

### Hyperparameter Tunning RSAST 

#### Accuracy: Subsequence Lenght Method 

In [10]:
filter_param=df_result[df_result.rpoint=='(lenthg ts)//2']
filter_param.rpoint.unique()

array(['(lenthg ts)//2'], dtype=object)

In [11]:
filter_param=filter_param[filter_param.nb_per_class=='(max instances per class)//2']
filter_param.nb_per_class.unique()

array(['(max instances per class)//2'], dtype=object)

In [12]:
# create a pivot table with the mean of score by dataset
len_method_vs_ds=pd.pivot_table(filter_param, values='accuracy', index=['len_method'],columns=['dataset_name'], aggfunc='mean')
len_method_vs_ds=np.transpose(len_method_vs_ds)#[['ACF','PACF']]
len_method_vs_ds=len_method_vs_ds.reset_index()
df_rocket=df_other_methods[df_other_methods["method"]=="ROCKET"]
merged_df = len_method_vs_ds.merge(df_rocket,left_on='dataset_name', right_on='dataset',  how='left')
merged_df=merged_df.drop('dataset',axis=1)
merged_df

,dataset_name,ACF,ACF&PACF,Max ACF,Max PACF,None,PACF,score,method
0,Chinatown,0.945578,0.971817,0.958017,0.971623,0.966375,0.974927,0.982507,ROCKET
1,Coffee,0.964286,1.000000,0.978571,0.935714,1.000000,1.000000,1.000000,ROCKET
2,DistalPhalanxOutlineAgeGroup,0.734772,0.724700,0.773621,0.729017,0.729496,0.717506,0.748201,ROCKET
3,DistalPhalanxOutlineCorrect,0.650000,0.713043,0.665942,0.700000,0.723913,0.723188,0.771739,ROCKET
4,GunPoint,0.830222,0.976000,0.845778,0.972444,0.965778,0.974667,1.000000,ROCKET
5,ItalyPowerDemand,0.949660,0.946615,0.952705,0.944866,0.951668,0.944866,0.968902,ROCKET
6,SmoothSubspace,0.870222,0.910667,0.879556,0.896889,0.876000,0.908889,0.980000,ROCKET
7,SonyAIBORobotSurface2,0.751242,0.899685,0.756348,0.866387,0.837356,0.893319,0.916055,ROCKET
8,SyntheticControl,0.920000,0.967333,0.896000,0.963333,0.981333,0.966667,1.000000,ROCKET


In [13]:

len_method_vs_ds=pd.pivot_table(filter_param, values='accuracy', index=['len_method'],columns=['dataset_name'], aggfunc='var')
len_method_vs_ds=np.transpose(len_method_vs_ds)#[['ACF','PACF']]
len_method_vs_ds=len_method_vs_ds.reset_index()
len_method_vs_ds

# Summary statistics for a Series (single variable)
researchpy.summary_cont(filter_param.groupby(['len_method'])['accuracy'], conf = 0.95)

,N,Mean,SD,SE,95% Conf.,Interval
len_method,,,,,,
ACF,105,0.8466,0.1003,0.0098,0.8272,0.8660
ACF&PACF,105,0.9033,0.0939,0.0092,0.8851,0.9215
Max ACF,105,0.8590,0.0923,0.0090,0.8411,0.8768
Max PACF,105,0.8925,0.0919,0.0090,0.8747,0.9103
None,105,0.8898,0.0952,0.0093,0.8713,0.9082
PACF,105,0.9015,0.0952,0.0093,0.8831,0.9200


In [14]:
stats = filter_param.groupby(['len_method'])['accuracy'].agg(['mean', 'count', 'std'])

ci95_hi = []
ci95_lo = []

for i in stats.index:
    m, c, s = stats.loc[i]
    ci95_hi.append(m + 1.96*s/math.sqrt(c))
    ci95_lo.append(m - 1.96*s/math.sqrt(c))

stats['ci95_hi'] = ci95_hi
stats['ci95_lo'] = ci95_lo
print(stats.head(10))

                mean  count       std   ci95_hi   ci95_lo
len_method                                               
ACF         0.846637    105  0.100295  0.865821  0.827453
ACF&PACF    0.903278    105  0.093930  0.921244  0.885311
Max ACF     0.858980    105  0.092317  0.876638  0.841322
Max PACF    0.892511    105  0.091907  0.910090  0.874931
None        0.889775    105  0.095220  0.907988  0.871561
PACF        0.901542    105  0.095184  0.919748  0.883335


#### Generate CD Diagram Tunning Hyperparameter

In [15]:
from cd_function import *
#generate hyperparameter tuning cd diagrams
for k,ints in enumerate(df_result.nb_per_class.unique()):
   ints_r=ints.replace("(","\(")
   ints_r=ints_r.replace(")","\)")
   for len_m in df_result.len_method.unique():
      df_to_cd=df_perf[df_perf.classifier_name.str.contains(r'^'+len_m+':.*nb_inst_per_class='+str(ints_r)+'$')]
      print('len_method'+str(len_m))
      print('nb_per_class'+str(ints))
      draw_cd_diagram(df_to_cd, labels=True, title=len_m +" comparison, nb_inst_per_class="+str(ints), name_file='images_cd_diagram/cd-diagram_'+len_m+str(k)+'.png')


len_methodACF&PACF
nb_per_class(max instances per class)//2


c:\Users\Surface pro\anaconda3\lib\site-packages\scipy\stats\_morestats.py:3255: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
c:\Users\Surface pro\anaconda3\lib\site-packages\scipy\stats\_morestats.py:3269: UserWarning: Sample size too small for normal approximation.
  warnings.warn("Sample size too small for normal approximation.")


len_methodACF
nb_per_class(max instances per class)//2
the null hypothesis over the entire classifiers cannot be rejected


c:\Users\Surface pro\anaconda3\lib\site-packages\scipy\stats\_morestats.py:3255: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
c:\Users\Surface pro\anaconda3\lib\site-packages\scipy\stats\_morestats.py:3269: UserWarning: Sample size too small for normal approximation.
  warnings.warn("Sample size too small for normal approximation.")


len_methodPACF
nb_per_class(max instances per class)//2
the null hypothesis over the entire classifiers cannot be rejected


...

c:\Users\Surface pro\anaconda3\lib\site-packages\scipy\stats\_morestats.py:3255: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
c:\Users\Surface pro\anaconda3\lib\site-packages\scipy\stats\_morestats.py:3269: UserWarning: Sample size too small for normal approximation.
  warnings.warn("Sample size too small for normal approximation.")


len_methodMax PACF
nb_per_class100
the null hypothesis over the entire classifiers cannot be rejected


c:\Users\Surface pro\anaconda3\lib\site-packages\scipy\stats\_morestats.py:3255: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
c:\Users\Surface pro\anaconda3\lib\site-packages\scipy\stats\_morestats.py:3269: UserWarning: Sample size too small for normal approximation.
  warnings.warn("Sample size too small for normal approximation.")


len_methodNone
nb_per_class100
the null hypothesis over the entire classifiers cannot be rejected


c:\Users\Surface pro\anaconda3\lib\site-packages\scipy\stats\_morestats.py:3255: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
c:\Users\Surface pro\anaconda3\lib\site-packages\scipy\stats\_morestats.py:3269: UserWarning: Sample size too small for normal approximation.
  warnings.warn("Sample size too small for normal approximation.")


In [16]:
_, average_ranks, _ = wilcoxon_holm(df_perf=df_perf)
min_rank= min(average_ranks)
average_ranks=pd.DataFrame(average_ranks)
average_ranks

c:\Users\Surface pro\anaconda3\lib\site-packages\scipy\stats\_morestats.py:3255: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
c:\Users\Surface pro\anaconda3\lib\site-packages\scipy\stats\_morestats.py:3269: UserWarning: Sample size too small for normal approximation.
  warnings.warn("Sample size too small for normal approximation.")


,0
Max ACF: n_random_points=1 nb_inst_per_class=1,214.888889
ACF: n_random_points=1 nb_inst_per_class=1,213.444444
Max ACF: n_random_points=10 nb_inst_per_class=1,207.166667
ACF: n_random_points=10 nb_inst_per_class=1,205.000000
Max PACF: n_random_points=1 nb_inst_per_class=1,204.500000
...,...
ACF&PACF: n_random_points= (lenthg ts)//2 nb_inst_per_class=30,55.500000
None: n_random_points=10 nb_inst_per_class=100,54.500000
ACF&PACF: n_random_points=100 nb_inst_per_class=(max instances per class)//2,53.500000
ACF&PACF: n_random_points= (lenthg ts)//2 nb_inst_per_class=50,51.388889


In [17]:
#generate example cd diagrams
#draw_cd_diagram(pd.read_csv('example_cd_diagram.csv'), labels=True, title="Example CD Diagram", name_file='cd-diagram_example.png')

#### Focus on most accurate lenght method

##### Overall Accuracy

In [18]:
# highest accurate hyperparameters
average_ranks[average_ranks[0]==min_rank][0].index

Index(['ACF&PACF: n_random_points=100 nb_inst_per_class=50'], dtype='object')

In [19]:
# filter by method with ACF&PACF
# create a pivot table with the mean of score by hyperparameter
df_result_acc=df_result[df_result["len_method"]=="ACF&PACF"]
pivot = pd.pivot_table(df_result_acc, values='accuracy', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')
pivot = pivot.reindex(columns=["(max instances per class)//2","1","10","30","50","100"], index=["(lenthg ts)//2","1","10","30","50","100"])
pivot

nb_per_class,(max instances per class)//2,1,10,30,50,100
rpoint,,,,,,
(lenthg ts)//2,0.903278,0.862845,0.892061,0.911307,0.913508,0.910002
1,0.890607,0.780525,0.867425,0.883828,0.872440,0.878069
10,0.899778,0.845827,0.882947,0.891565,0.892031,0.890808
30,0.909236,0.860804,0.880145,0.889029,0.893775,0.894572
50,0.904643,0.858797,0.883900,0.894106,0.892872,0.890225
100,0.910622,0.860812,0.878514,0.891604,0.897336,0.893455


In [20]:
# create a pivot table with the variatioon of score by hyperparameter
pd.pivot_table(df_result_acc, values='accuracy', index=['rpoint'],columns=['nb_per_class'], aggfunc='var')

nb_per_class,(max instances per class)//2,1,10,100,30,50
rpoint,,,,,,
(lenthg ts)//2,0.008823,0.007882,0.009996,0.009603,0.009711,0.009809
1,0.006761,0.016703,0.008075,0.009543,0.009266,0.010421
10,0.009507,0.007888,0.010715,0.012018,0.011682,0.011821
100,0.008307,0.009146,0.013198,0.011479,0.012070,0.011317
30,0.008067,0.010218,0.012187,0.011707,0.012209,0.011785
50,0.008662,0.008551,0.012015,0.013223,0.011196,0.012026


##### Overall time complexity

In [21]:
# create a pivot table with the mean of time spent in minutes
pivot=pd.pivot_table(df_result_acc, values=['time','cweights_time','fsubsequence_time','tdataset_time','tclassifier_time'], index=['dataset_name'], aggfunc='mean')/60
pivot = pivot.reindex(columns=['cweights_time','fsubsequence_time','tdataset_time','tclassifier_time','time'])
pivot

,cweights_time,fsubsequence_time,tdataset_time,tclassifier_time,time
dataset_name,,,,,
Chinatown,0.000148,0.001009,0.001189,0.000235,0.002611
Coffee,0.001126,0.082135,2.156343,0.000738,2.240458
DistalPhalanxOutlineAgeGroup,0.000529,0.020612,6.829225,0.015880,6.866520
DistalPhalanxOutlineCorrect,0.000491,0.015808,8.404571,0.010732,8.431886
GunPoint,0.000645,0.027494,1.169863,0.000965,1.199090
ItalyPowerDemand,0.000136,0.001336,0.009811,0.000842,0.012150
SmoothSubspace,0.000103,0.002251,0.010263,0.002864,0.015516
SonyAIBORobotSurface2,0.000321,0.002770,0.030471,0.000283,0.033882
SyntheticControl,0.000596,0.022403,3.277018,0.007352,3.307617


In [22]:
# create a pivot table with the mean of time spent in minutes by random points and instances per class
pivot = pd.pivot_table(df_result_acc, values='time', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')/60
pivot = pivot.reindex(columns=["(max instances per class)//2","1","10","30","50","100"], index=["(lenthg ts)//2","1","10","30","50","100"])
pivot

nb_per_class,(max instances per class)//2,1,10,30,50,100
rpoint,,,,,,
(lenthg ts)//2,3.967635,0.087849,0.857774,2.079303,2.785928,4.249620
1,0.136709,0.016466,0.059792,0.075305,0.101487,0.159058
10,0.967801,0.053073,0.202243,0.557632,0.830703,1.406532
30,2.847241,0.065659,0.596354,1.667156,2.483686,4.211959
50,4.549979,0.102932,0.936347,2.631405,3.928274,6.609370
100,5.634172,0.127002,1.282955,3.432775,5.027691,8.288030


##### Calculate weights time complexity

In [23]:
pd.pivot_table(df_result_acc, values='cweights_time', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,(max instances per class)//2,1,10,100,30,50
rpoint,,,,,,
(lenthg ts)//2,0.022648,0.025738,0.027714,0.020942,0.026246,0.023184
1,0.029875,0.022034,0.022386,0.023180,0.024261,0.023495
10,0.022351,0.022504,0.021176,0.020796,0.018380,0.023092
100,0.021229,0.022295,0.019622,0.023922,0.021289,0.024802
30,0.024764,0.025776,0.022052,0.023253,0.021224,0.023302
50,0.026148,0.019950,0.020483,0.023775,0.017310,0.023802


##### Finding subsequences time complexity

In [24]:
pd.pivot_table(df_result_acc, values='fsubsequence_time', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,(max instances per class)//2,1,10,100,30,50
rpoint,,,,,,
(lenthg ts)//2,1.048488,0.088898,0.710858,1.471322,1.267152,1.433057
1,1.561904,0.143997,0.456703,1.035440,0.785233,0.892262
10,0.937482,0.040728,0.367889,1.079674,0.667620,0.832107
100,0.987144,0.048964,0.436373,1.402857,0.923808,1.219290
30,0.921290,0.047483,0.431415,1.250624,0.801752,1.023453
50,1.062443,0.041908,0.446750,1.365893,0.859830,1.062731


##### Transform Dataset time complexity

In [25]:
pd.pivot_table(df_result_acc, values='tdataset_time', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,(max instances per class)//2,1,10,100,30,50
rpoint,,,,,,
(lenthg ts)//2,236.753083,4.981848,50.654668,253.228274,123.302976,165.476751
1,6.360697,0.360298,1.444127,8.440775,3.540797,5.127611
10,57.021101,1.383745,11.697447,83.180543,32.706855,48.903206
100,336.754064,7.422130,76.427129,495.448703,204.808242,300.116780
30,169.695379,3.665377,35.257446,251.183977,99.082785,147.804205
50,271.641805,5.975791,55.629795,394.823717,156.841683,234.384575


##### Classifier time complexity

In [26]:
pd.pivot_table(df_result_acc, values='tclassifier_time', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,(max instances per class)//2,1,10,100,30,50
rpoint,,,,,,
(lenthg ts)//2,0.219153,0.173013,0.068219,0.239921,0.151372,0.206802
1,0.244793,0.460037,1.661260,0.042483,0.165435,0.044047
10,0.083711,1.735639,0.045850,0.107382,0.062866,0.081411
100,0.276140,0.124746,0.091333,0.391278,0.205032,0.289695
30,0.185353,0.198990,0.066816,0.251335,0.118868,0.164110
50,0.256786,0.135885,0.080715,0.336213,0.158857,0.216699
